In [1]:
# usage: py gen.py
# creates all.data file which contains sequence of shift reduce actions

import os
from utils import *
import time
import pickle

# indexes words in sentence to prevent duplicates
def idx_tree(root, i = 0, star = 0):
    if not root.l and not root.r:
        if "*" not in root.label:
            root.label += "/" + str(i)
            i += 1
        else:
            root.label += "/" + str(star)
            star +=1
        return (root, i, star)
    if root.l:
        (root.l, i, star) = idx_tree(root.l, i, star)
    if root.r:
        (root.r, i, star) = idx_tree(root.r, i, star)
    return (root, i, star)

# create stack/buffer actions from sentence and tree
def generate_actions(t, s):

    def match_tree(t1, t2): # subroutine for checking matching trees
        if t1 is None and t2 is None: # base case
            return True
        if t1 is not None and t2 is not None:
            return (t1.label == t2.label) and \
            match_tree(t1.l, t2.l) and \
            match_tree(t1.r, t2.r)
        return False

    def binary_label_dfs(root, s0, s1): # subroutine for determining the label

        if match_tree(root.l, s0) and match_tree(root.r, s1): # base case
            return root.label
        if root.l:
            left_label = binary_label_dfs(root.l, s0, s1)
            if left_label:
                return left_label
        if root.r:
            right_label = binary_label_dfs(root.r, s0, s1)
            return right_label if right_label else None

    def unary_label_dfs(root, s0): # subroutine for determining the label
        if match_tree(root.l, s0) and root.r == None: # base case
            return root.label

        # recursive calls
        child_label = None
        if root.l:
            child_label = unary_label_dfs(root.l, s0)

        if root.r and not child_label:
            child_label = unary_label_dfs(root.r, s0)

        if child_label:
            return child_label

    ret = []
    buff = list(map(Node, s.split()[::-1])) # reverse sentence for O(1) pop
    stack = []

    while buff or len(stack) > 1: # end when buffer consumed & stack has tree
        # print(stack_to_str(stack))

        # write the stack and buffer before action is performed
        st = list(map(lambda x: tree_to_str(x), stack))
        bu = list(map(lambda x: tree_to_str(x), buff[::-1]))
        final_label = ""
        final_action = ""
        templab = 'NOTHING'
        # try to reduce top two items
        if len(stack) > 1: # reduce
            left = stack[len(stack) - 2]
            right = stack[len(stack) - 1]
            new_node = Node(binary_label_dfs(t, left, right))
            if new_node.label: # found a matching reduce
                final_label = new_node.label

                #TEMP
                lab = final_label.replace('=', '-').split('-')
                templab = lab
                if lab[0]: final_label = lab[0]
                else: final_label = lab[1]

                final_action = "binary"
                new_node.r = stack.pop()
                new_node.l = stack.pop()
                stack.append(new_node)
            else: # try to unary reduce
                child = stack[len(stack) - 1]
                new_node = Node(unary_label_dfs(t, child))
                if new_node.label: # found a unary reduce
                    final_label = new_node.label

                    #TEMP
                    lab = final_label.replace('=', '-').split('-')
                    templab = lab
                    if lab[0]: final_label = lab[0]
                    else: final_label = lab[1]

                    final_action = "unary"
                    new_node.l = stack.pop()
                    stack.append(new_node)
                else: # shift
                    final_action = "shift"
                    if "*" in buff[-1].label:
                        final_action += " star"
                    stack.append(buff.pop())
        elif len(stack) == 1: # just try unary reduce
            child = stack[len(stack) - 1]
            new_node = Node(unary_label_dfs(t, child))
            if new_node.label: # found a unary reduce
                final_label = new_node.label

                #TEMP
                lab = final_label.replace('=', '-').split('-')
                templab = lab
                if lab[0]: final_label = lab[0] # for cases like -PRT-
                else: final_label = lab[1]

                final_action = "unary"
                new_node.l = stack.pop()
                stack.append(new_node)
            else: # shift
                final_action = "shift"
                if "*" in buff[-1].label:
                        final_action += " star"
                stack.append(buff.pop())
        else: # shift
            final_action = "shift"
            if "*" in buff[-1].label:
                        final_action += " star"
            stack.append(buff.pop())

        # append all changes
        act = final_action
        lab = final_label
        if lab == '' and templab != 'NOTHING': print(templab)

        if act == 'shift' or act == 'shift star': ret.append(datum(st, bu, act))
        else: ret.append(datum(st, bu, act + " " + lab))

        # if act + " " + lab == 'unary ': print('what the fuck')
    return ret


if __name__ == '__main__':
    t0 = time.time()
    treepath = "../treebank/treebank_3/parsed/mrg/wsj/"
    outpath = "../data/"

    # open file and save as one large string
    text = ""
    for folder in os.listdir(treepath):
        if folder.startswith('.'):
            continue
        for filename in os.listdir(treepath + folder):
            if filename.startswith('.'):
                continue
            with open(treepath + folder + "/" + filename, 'r') as f:
                text += f.read().replace('\n', '')
        #break # test only one folder for speed

    tree_string_list = []
    s = []
    start = 0
    for i in range(len(text)):
        if text[i] == "(":
            s.append("(")
        elif text[i] == ")":
            s.pop()
            if not s:
                tree_string_list.append(text[start : i + 1])
                start = i + 1


    # turn tree strings into tree_list
    tree_list = []
    for t in tree_string_list:
        tree_list.append((parse_tree(t[1:-1])))

    tree_list = list(map(lambda x: idx_tree(x)[0], tree_list))

    # use inorder traveral to generate sentences from trees
    sentences = []
    for t in tree_list:
        sentences.append(inorder_sentence(t).lstrip()) # extra space on left

    idx = 0
    output_list = []
    with open(outpath + 'actions.data', 'wb') as f:
        for t, s in zip(tree_list[1:], sentences[1:]):
            dat = generate_actions(t, s)
            output_list.extend(dat)
            if idx % 100 == 0: print(str(idx) + "..." , end = ' ', flush = True)
            idx += 1
            if idx == 10000: break # cut the test data short
        print()
        pickle.dump(output_list, f)

    t1 = time.time()
    total = t1 - t0
    print("runtime: " + str(total))


0... 100... 200... 300... 400... 500... 600... 700... 800... 900... 1000... 1100... 1200... 1300... 1400... 1500... 1600... 1700... 1800... 1900... 2000... 2100... 2200... 2300... 2400... 2500... 2600... 2700... 2800... 2900... 3000... 3100... 3200... 3300... 3400... 3500... 3600... 3700... 3800... 3900... 4000... 4100... 4200... 4300... 4400... 4500... 4600... 4700... 4800... 4900... 5000... 5100... 5200... 5300... 5400... 5500... 5600... 5700... 5800... 5900... 6000... 6100... 6200... 6300... 6400... 6500... 6600... 6700... 6800... 6900... 7000... 7100... 7200... 7300... 7400... 7500... 7600... 7700... 7800... 7900... 8000... 8100... 8200... 8300... 8400... 8500... 8600... 8700... 8800... 8900... 9000... 9100... 9200... 9300... 9400... 9500... 9600... 9700... 9800... 9900... 
runtime: 262.895879983902


In [2]:
from utils import *
import pickle

# feature list for one action:
# 0: ground truth action
#----------------------------------------
# 1-4: top four words on buffer, <null> if they don't exist
#----------------------------------------
# 5: label of stack[0] ("<word>" if word)
# 6: word of stack[0] ("<label>" if constituent label)
# 7-10: leftmost POS and word, rightmost POS and word
#----------------------------------------
# 11: label of stack[1] ("<word>" if word)
# 12: word of stack[1] ("<label>" if constituent label)
# 13-16: leftmost POS and word, rightmost POS and word
#----------------------------------------
# 17: label of stack[2] ("<word>" if word)
# 18: word of stack[2] ("<label>" if constituent label)
# 19-22: leftmost POS and word, rightmost POS and word
#----------------------------------------
# 23: label of stack[3] ("<word>" if word)
# 24: word of stack[3] ("<label>" if constituent label)
# 25-28: leftmost POS and word, rightmost POS and word

def rearrange(f):
    # 0(5): label of stack[0] ("<word>" if word)
    # 1(7): leftmost POS
    # 2(9): rightmost POS
    # 3(11): label of stack[1] ("<word>" if word)
    # 4(13): leftmost POS
    # 5(15): rightmost POS
    # 6(17): label of stack[2] ("<word>" if word)
    # 7(19): leftmost POS
    # 8(21): rightmost POS
    # 9(23): label of stack[3] ("<word>" if word)
    # 10(25): leftmost POS
    # 11(27): rightmost POS
    labels = set([5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27])
    new_list = []
    for i in range(1, len(f)):
        if i in labels:
            new_list.append(f[i])
    for i in range(1, len(f)):
        if i not in labels: # and i not in prefix:
            new_list.append(f[i])
    new_list.append(f[0]) # append label to end
    return new_list

def get_left(t):
    if not t.l.r and not t.l.l: # if l child is unary
        return [t.label, unindex(t.l.label)]
    else:
        return get_left(t.l)

def get_right(t):
    if not t.l.r and not t.l.l: # if l child is unary
        return [t.label, unindex(t.l.label)]
    else:
        if t.r: # handle strange case of "(NP (NNP Moscow) ))"
            return get_right(t.r)
        else:
            return get_right(t.l)

def unindex(a):
    return a.split("/")[0].rstrip().lstrip() # assume no words contain "/"

def extract_features(d):
    features = []
    stack = d.stack[::-1]
    buff = d.buff

    # top four buffer words
    for i in range(0,4):
        if len(buff) > i:
            features.append(unindex(buff[i]))
        else:
            features.append("<null>")

        # stack items
    for i in range(0,4):
        if len(stack) > i:
            tree = parse_tree(stack[i])
            if tree.l or tree.r: # label
                features.append(tree.label)
                features.append("<label>")
            else: # word
                features.append("<word>")
                features.append(unindex(tree.label))


            if tree.l and tree.r: # binary rule
                # assume a depth of 3 at least
                features.extend(get_left(tree.l))
                features.extend(get_right(tree.r))
            else:
                features.extend(["<null>"] * 4)
        else:
            features.extend(["<null>"] * 6)

    return features



if __name__ == '__main__':
    datapath = "../data/actions.data"
    outpath = "../data/"
    debug = open("feat_labels.log", "w")

    # open file and save as one large string
    data_list = pickle.load(open(datapath, 'rb'))

    final_list = [] # list of lists of features
    final_list_read = [] # list of lists of features for debugging
    i = 0
    for d in data_list:
        # if not d: continue# gets rid of weird empty string error

        features  = [((d.label.split("-")[0]).split('_')[0]).split('=')[0]] + extract_features(d)
        final_list.append(rearrange(features))
        final_list_read.append(features)
        if i % 10000 == 0: print(str(i) + '...', end='', flush=True)
        i += 1

    print("Feature vectors: " + str(len(final_list)))

    with open(outpath + "train.data", "wb") as f:
        pickle.dump(final_list[10000:], f)

    with open(outpath + "test.data", "wb") as f:
        pickle.dump(final_list[:10000], f)

    with open(outpath + "all_features.data", "wb") as f:
        pickle.dump(final_list, f)

    # write in readable form
    i = 1
    with open(outpath + "features_read.txt", "w") as f:
        for fl1, fl2 in zip(final_list, final_list_read):
            f.write(str(fl1[:12]) + "\n")
            f.write(str(fl1[12:]) + "\n\n")

            f.write(str(fl2[0:1]) + "\n")
            f.write(str(fl2[1:5]) + "\n")
            f.write(str(fl2[5:11]) + "\n")
            f.write(str(fl2[11:17]) + "\n")
            f.write(str(fl2[17:23]) + "\n")
            f.write(str(fl2[23:29]) + "\n")
            i += 1
            if i == 5000: break


0...10000...20000...30000...40000...50000...60000...70000...80000...90000...100000...110000...120000...130000...140000...150000...160000...170000...180000...190000...200000...210000...220000...230000...240000...250000...

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/awl2144/.virtualenvs/test_py3/lib/python3.4/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-d8b5b0d7aa59>", line 118, in <module>
    features  = [((d.label.split("-")[0]).split('_')[0]).split('=')[0]] + extract_features(d)
  File "<ipython-input-2-d8b5b0d7aa59>", line 75, in extract_features
    features.append(unindex(buff[i]))
  File "<ipython-input-2-d8b5b0d7aa59>", line 65, in unindex
    return a.split("/")[0].rstrip().lstrip() # assume no words contain "/"
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/awl2144/.virtualenvs/test_py3/lib/python3.4/site-packages/IPython/core/interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

Dur

KeyboardInterrupt: 

In [ ]:
class NetProperties:
    def __init__(self, word_embed_dim, pos_embed_dim, hidden_dim, minibatch_size):
        self.word_embed_dim = word_embed_dim
        self.pos_embed_dim = pos_embed_dim
        self.hidden_dim = hidden_dim
        self.minibatch_size = minibatch_size
        
import dynet as dynet
import random
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import numpy as np
import pickle

class Network:
    def __init__(self, vocab, properties):
        self.properties = properties
        self.vocab = vocab

        # first initialize a computation graph container (or model).
        self.model = dynet.Model()

        # assign the algorithm for backpropagation updates.
        self.updater = dynet.AdamTrainer(self.model)

        # create embeddings for words and tag features.
        self.word_embedding = self.model.add_lookup_parameters((vocab.num_words(), properties.word_embed_dim))
        self.tag_embedding = self.model.add_lookup_parameters((vocab.num_tag_feats(), properties.pos_embed_dim))

        # assign transfer function
        self.transfer = dynet.rectify  # can be dynet.logistic or dynet.tanh as well.

        # define the input dimension for the embedding layer.
        # here we assume to see two words after and before and current word (meaning 5 word embeddings)
        # and to see the last two predicted tags (meaning two tag embeddings)
        self.input_dim = 15 * properties.word_embed_dim + 12 * properties.pos_embed_dim

        # define the hidden layer.
        self.hidden_layer = self.model.add_parameters((properties.hidden_dim, self.input_dim))

        # define the hidden layer bias term and initialize it as constant 0.2.
        self.hidden_layer_bias = self.model.add_parameters(properties.hidden_dim, init=dynet.ConstInitializer(0.2))

        # define the output weight.
        self.output_layer = self.model.add_parameters((vocab.num_tags(), properties.hidden_dim))

        # define the bias vector and initialize it as zero.
        self.output_bias = self.model.add_parameters(vocab.num_tags(), init=dynet.ConstInitializer(0))

    def build_graph(self, features):
        # extract word and tags ids
        word_ids = [self.vocab.word2id(word_feat) for word_feat in features[12:-1]]
        tag_ids = [self.vocab.feat_tag2id(tag_feat) for tag_feat in features[0:12]]

        # extract word embeddings and tag embeddings from features
        word_embeds = [self.word_embedding[wid] for wid in word_ids]
        tag_embeds = [self.tag_embedding[tid] for tid in tag_ids]

        # concatenating all features (recall that '+' for lists is equivalent to appending two lists)
        embedding_layer = dynet.concatenate(word_embeds + tag_embeds)

        # calculating the hidden layer
        # .expr() converts a parameter to a matrix expression in dynet (its a dynet-specific syntax).
        hidden = self.transfer(self.hidden_layer.expr() * embedding_layer + self.hidden_layer_bias.expr())

        # calculating the output layer
        output = self.output_layer.expr() * hidden + self.output_bias.expr()

        # return the output as a dynet vector (expression)
        return output

    def train(self, train_file, epochs):
        plot_on = False
        # matplotlib config
        loss_values = []
        if plot_on:
            plt.ion()
            ax = plt.gca()
            ax.set_xlim([0, 10])
            ax.set_ylim([0, 5])
            plt.title("Loss over time")
            plt.xlabel("Minibatch")
            plt.ylabel("Loss")

        for i in range(epochs):
            print('started epoch', (i+1))
            losses = []
            train_data = pickle.load(open( "../data/train.data", "rb" ))

            # shuffle the training data.
            random.shuffle(train_data)

            step = 0
            for fl in train_data:
                features, label = fl[:-1], fl[-1]
                gold_label = self.vocab.tag2id(label)
                result = self.build_graph(features)

                # getting loss with respect to negative log softmax function and the gold label.
                loss = dynet.pickneglogsoftmax(result, gold_label)

                # appending to the minibatch losses
                losses.append(loss)
                step += 1

                if len(losses) >= self.properties.minibatch_size:
                    # now we have enough loss values to get loss for minibatch
                    minibatch_loss = dynet.esum(losses) / len(losses)

                    # calling dynet to run forward computation for all minibatch items
                    minibatch_loss.forward()

                    # getting float value of the loss for current minibatch
                    minibatch_loss_value = minibatch_loss.value()

                    # printing info and plotting
                    loss_values.append(minibatch_loss_value)
                    if len(loss_values)%10==0:
                        if plot_on:
                            ax.set_xlim([0, len(loss_values)+10])
                            ax.plot(loss_values)
                            plt.draw()
                            plt.pause(0.0001)
                        progress = round(100 * float(step) / len(train_data), 2)
                        print('current minibatch loss', minibatch_loss_value, 'progress:', progress, '%')

                    # calling dynet to run backpropagation
                    minibatch_loss.backward()

                    # calling dynet to change parameter values with respect to current backpropagation
                    self.updater.update()

                    # empty the loss vector
                    losses = []

                    # refresh the memory of dynet
                    dynet.renew_cg()

            # there are still some minibatch items in the memory but they are smaller than the minibatch size
            # so we ask dynet to forget them
            dynet.renew_cg()

    def decode(self, features):

        # running forward
        output = self.build_graph(features)

        # getting list value of the output
        scores = output.npvalue()

        # getting best tag
        best_tag_id = np.argmax(scores)

        # assigning the best tag
        pred = self.vocab.tagid2tag_str(best_tag_id)

        # refresh dynet memory (computation graph)
        dynet.renew_cg()

        return pred

    def load(self, filename):
        self.model.populate(filename)

    def save(self, filename):
        self.model.save(filename)


In [ ]:
import pickle
from net_properties import *
from utils import *
from network import *
import time



if __name__ == '__main__':
    t0 = time.time()

    we = 100
    pe = 50
    hidden = 200
    minibatch = 1000
    epochs = 5

    net_properties = NetProperties(we, pe, hidden, minibatch)
    vocab = Vocab("../data/train.data")
    pickle.dump((vocab, net_properties), open("../data/vocab_net.data", 'wb'))
    network = Network(vocab, net_properties)
    network.train("../data/train.data", epochs)
    network.save("../networks/net.model")

    t1 = time.time()
    total = t1 - t0
    print("runtime: " + str(total))

